In [170]:
!pip install pyphen


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [171]:
import pandas as pd
import nltk
nltk.download('punkt')  
nltk.download('stopwords')
import pyphen
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/damodargupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/damodargupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [172]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import os
import re


In [173]:
df = pd.read_excel('/Users/damodargupta/Desktop/blackcoffer-project/Input.xlsx')
df

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [178]:
# some url does not exist - 404 error
# dropping those urls from the dataframe

df = df[df['URL_ID'] != 11668]
df = df[df['URL_ID'] != 17671.4]


In [180]:
def extract_title(url):
    uClient = uReq(url)
    Page = uClient.read()
    uClient.close()
    url_html = bs(Page, "html.parser")
    box = url_html.findAll("h1", {"class": "entry-title"})
   
    if box == []:
        box = url_html.findAll("h1", {"class": "tdb-title-text"})
    
    if box != []:
        box = str(box)
        soup = bs(box, "html.parser")
        title = soup.h1.get_text()
        return title


In [181]:
def extract_text(url):
    uClient = uReq(url)
    Page = uClient.read()
    uClient.close()
    url_html = bs(Page, "html.parser")
    box = url_html.findAll("div", {"class": "td-post-content tagdiv-type"})
    
    if box == []:
        box = url_html.findAll("div", {"class": "tdb-block-inner td-fix-index"})
        
    if box != []:
        box = str(box)
        soup = bs(box, 'html.parser')
        elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7','h8', 'h9', 'h10', 'ol' , 'ul'])
        extracted_text = ''
        for element in elements:
            extracted_text += element.get_text() + '\n'
        
        return extracted_text
    
    

In [182]:
# Define a function to save text to a file
def save_text_to_file(url_id, text):
    with open(f"{url_id}.txt", "w", encoding="utf-8") as file:
        file.write(text)

# Loop through the DataFrame and save text to files
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    title = extract_title(url)
    text = extract_text(url)

    if title is not None and text is not None:
        # Concatenate the title and text
        result = title + '\n' + '\n' + text
    
        # Save both the title and text to the same file
        with open("/Users/damodargupta/Desktop/blackcoffer-project/saved_articles/" + f"{url_id}.txt", "w", encoding="utf-8") as file:
            file.write(result)
        print(f"Saved title and text from URL {url} to {url_id}.txt")

Saved title and text from URL https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/ to 123.0.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/ to 321.0.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/ to 2345.0.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/ to 4321.0.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/ to 432.0.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/ to 2893.8.txt
Saved title and text from URL https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030/ to 3355.6.txt
Saved title and text from

In [ ]:
extract_title("https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/")
extract_text("https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/")

[]


AttributeError: 'NoneType' object has no attribute 'get_text'

In [30]:
def read_stop_words(file_path):
    with open(file_path, 'r') as stop_words_file:
        stop_words = stop_words_file.read().splitlines()
    return stop_words

def cleaning(url):
    stopwords_folder = '/Users/damodargupta/Desktop/blackcoffer-project/StopWords'

    stop_words = []

    stop_word_files = [
        'StopWords_Auditor.txt',
        'StopWords_Currencies.txt',
        'StopWords_DatesandNumbers.txt',
        'StopWords_Generic.txt',
        'StopWords_GenericLong.txt',
        'StopWords_Geographic.txt',
        'StopWords_Names.txt'
    ]

    for file_name in stop_word_files:
        file_path = os.path.join(stopwords_folder, file_name)
        stop_words.extend(read_stop_words(file_path))

    article_text = extract_text(url)
    words = str(article_text).split()

    cleaned_words = [word for word in words if word.lower() not in stop_words]

    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text


In [31]:
cleaning("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

'afraid robots jobs nurses, doctors, healthcareprofessionals? scared artificial intelligence control world acouple years?COVID-19 revealed vulnerable health care industry change structural technological transformation. COVID-19 pandemic unwelcome reminder health matters individuals society global economy. Improved health accounted one-third GDP capita growth developed economies past century. traditional boundaries industry dissolve roles emerge future health exponential innovation propels industry 2040. 2040, health care today longer exist. fundamental shift “health care” “health.” disease eliminated, science, data, technology, identify earlier, intervene proactively, understand progression consumers effectively actively sustain well-being. future focused wellness managed companies assume roles drive transformed health ecosystem. rate technological evolution continues speed up, companies ahead proactively plan future. healthcare industry, particular, pressing compounded pace declining 

In [75]:
def read_text_file(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        return None
    
# Tokenize text into words
def tokenize_text(text):
    # You can use more advanced tokenization techniques as needed
    return text.split()

# Create positive and negative dictionaries
def categorize_words(url):
    words = tokenize_text(cleaning(url))
    
    positive_dict = {}
    negative_dict = {}

    # Path to positive and negative word files
    positive_dict_file = read_text_file('/Users/damodargupta/Desktop/blackcoffer-project/MasterDictionary/positive-words.txt')
    negative_dict_file = read_text_file('/Users/damodargupta/Desktop/blackcoffer-project/MasterDictionary/negative-words.txt')
    
    for word in words:
        word = word.lower()  # Normalize to lowercase
        if word in positive_dict_file:
            positive_dict[word] = 'positive'
        elif word in negative_dict_file:
            negative_dict[word] = 'negative'
        
    # # Print the positive and negative dictionaries
    # print("Positive Dictionary:")
    # for word, sentiment in positive_dict.items():
    #     print(f"{word}: {sentiment}")

    # print("\nNegative Dictionary:")
    # for word, sentiment in negative_dict.items():
    #     print(f"{word}: {sentiment}")
    
    return positive_dict, negative_dict


In [76]:
categorize_words("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

({'intelligence': 'positive',
  'world': 'positive',
  'vulnerable': 'positive',
  'health': 'positive',
  'care': 'positive',
  'improved': 'positive',
  'future': 'positive',
  'innovation': 'positive',
  'understand': 'positive',
  'effectively': 'positive',
  'actively': 'positive',
  'sustain': 'positive',
  'managed': 'positive',
  'rate': 'positive',
  'evolution': 'positive',
  'speed': 'positive',
  'pace': 'positive',
  'ray': 'positive',
  'profound': 'positive',
  'tend': 'positive',
  'worth': 'positive',
  'promote': 'positive',
  'economic': 'positive',
  'die': 'positive',
  'loved': 'positive',
  'early': 'positive',
  'improve': 'positive',
  'quality': 'positive',
  'life': 'positive',
  'toll': 'positive',
  'healthy': 'positive',
  'access': 'positive',
  'improving': 'positive',
  'found': 'positive',
  'tremendous': 'positive',
  'benefits': 'positive',
  'safe': 'positive',
  'lower': 'positive',
  'grow': 'positive',
  'live': 'positive',
  'long': 'positive',


In [87]:
import nltk
from nltk.tokenize import word_tokenize

def calculate_score(url) : 
    positive_dict , negative_dict = categorize_words(url) 
    text = extract_text(url)

    # Tokenize the text using nltk
    tokens = word_tokenize(text)
    print(tokens)

    # Initialize the positive score
    positive_score = 0
    negative_score = 0

    # Calculate the positive score
    for token in tokens:
        if token.lower() in positive_dict:
            positive_score += 1
        elif token.lower() in negative_dict:
            negative_score -= 1
    
    negative_score = abs(negative_score)
    
    # Calculate the Polarity Score
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    
    # Calculate the subjectivity Score
    total_words_after_cleaning = len(cleaning(url).split())
    subjectivity_score = (positive_score + negative_score) / (total_words_after_cleaning + 0.000001)


    # Print the scores
    print("Positive Score:", positive_score)
    print("Negative Score:", negative_score)
    print("Polarity Score:", polarity_score)
    print("Subjectivity Score", subjectivity_score)


In [88]:
calculate_score("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

['Are', 'you', 'afraid', 'that', 'robots', 'will', 'take', 'over', 'the', 'jobs', 'of', 'nurses', ',', 'doctors', ',', 'and', 'other', 'healthcareprofessionals', '?', 'Are', 'you', 'scared', 'that', 'artificial', 'intelligence', 'will', 'control', 'the', 'world', 'within', 'acouple', 'of', 'years', '?', 'COVID-19', 'has', 'revealed', 'how', 'vulnerable', 'the', 'health', 'care', 'industry', 'is', 'to', 'change', 'and', 'its', 'need', 'for', 'structural', 'and', 'technological', 'transformation', '.', 'The', 'COVID-19', 'pandemic', 'has', 'been', 'an', 'unwelcome', 'reminder', 'of', 'just', 'how', 'much', 'health', 'matters', 'not', 'only', 'to', 'individuals', 'and', 'society', 'but', 'also', 'to', 'the', 'global', 'economy', '.', 'Improved', 'health', 'accounted', 'for', 'about', 'one-third', 'of', 'the', 'overall', 'GDP', 'per', 'capita', 'growth', 'of', 'developed', 'economies', 'in', 'the', 'past', 'century', '.', 'The', 'traditional', 'boundaries', 'of', 'the', 'industry', 'will',

In [102]:
# analysis of readability

def analysis_of_readiblity(url): 
    article = extract_text(url)

    sentences = sent_tokenize(article)
    words = word_tokenize(article)

    num_sentences = len(sentences)
    num_words = len(words)

    average_sentence_length = num_words / num_sentences
    
    # Create a syllable counter using the 'pyphen' library
    dic = pyphen.Pyphen(lang='en')

    # Define a function to count syllables in a word
    def count_syllables(word):
        return len(dic.inserted(word).split('-'))

    # Count the number of syllables for each word and determine complex words
    complex_words = [word for word in words if count_syllables(word) > 2]

    # Count the number of complex words
    num_complex_words = len(complex_words)

    percentage_complex_words = num_complex_words / num_words

    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    
    average_words_per_sentence = num_words / num_sentences

    print("Average Sentence Length:", average_sentence_length)
    print("Percentage of Complex Words:", percentage_complex_words)
    print("Fog Index:", fog_index)
    print("Average number of words per sentence:" , average_words_per_sentence)
    print('Number of complex words' , num_complex_words)



In [103]:
analysis_of_readiblity("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

Average Sentence Length: 32.63157894736842
Percentage of Complex Words: 0.15161290322580645
Fog Index: 13.113276740237689
Average number of words per sentence: 32.63157894736842
Number of complex words 188


In [107]:
def clean_word(word):
    return word.translate(str.maketrans('', '', string.punctuation)).lower()

def word_count(url):
    article = extract_text(url)
    words = word_tokenize(article)
    stop_words = set(stopwords.words('english'))
    cleaned_words = [clean_word(word) for word in words if clean_word(word) not in stop_words]
    total_cleaned_words = len(cleaned_words)
    print("Total Cleaned Words:", total_cleaned_words)

In [108]:
word_count("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

Total Cleaned Words: 773


In [114]:
def count_syllables(word):
    # Remove common suffixes that don't contribute to syllable count
    word = re.sub(r'[aeiouyAEIOUY]{2,}', 'a', word) 
    word = re.sub(r'[eE][sS]$', '', word)  
    word = re.sub(r'[eE][dD]$', '', word)  

    # Count the number of remaining vowels
    syllables = len(re.findall(r'[aeiouyAEIOUY]+', word))
    return max(1, syllables)  # Ensure at least 1 syllable

def syllable_per_word(url):
    article = extract_text(url)
    words = article.split()  
    syllable_counts = [count_syllables(word) for word in words]
    total_syllables = sum(syllable_counts)
    total_words = len(words)

    average_syllables_per_word = total_syllables / total_words
    rounded_average = round(average_syllables_per_word)

    print("Average Syllables Per Word:", rounded_average)
    

In [115]:
syllable_per_word("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

Average Syllables Per Word: 2


In [116]:
def count_personal_pronouns(url):
    article = extract_text(url)
    # Define the regular expression pattern to match personal pronouns
    pronoun_pattern = r'\b(I|we|my|ours|us)\b'

    # Use regex to find and count personal pronouns, excluding "US"
    personal_pronouns = re.findall(pronoun_pattern, article, flags=re.IGNORECASE)
    personal_pronouns = [pronoun for pronoun in personal_pronouns if pronoun.lower() != 'us']
    pronoun_count = len(personal_pronouns)

    print("Personal Pronouns Count:", pronoun_count)


In [117]:
count_personal_pronouns("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

Personal Pronouns Count: 7


In [118]:
def average_word_length(url):
    article = extract_text(url)
    words = article.split()  
    total_characters = sum(len(word) for word in words)
    total_words = len(words)
    average_word_length = total_characters / total_words
    print("Average Word Length:", average_word_length)


In [119]:
average_word_length("https://insights.blackcoffer.com/an-outlook-of-healthcare-by-the-year-2040-and-how-it-will-impact-human-lives/")

Average Word Length: 5.5009174311926605
